In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# 一个简单的神经网络，不包含残差连接
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(10, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# 初始化模型、损失函数和优化器
model_no_resnet = SimpleNN()
criterion = nn.MSELoss()
optimizer = optim.SGD(model_no_resnet.parameters(), lr=0.01)

# 生成随机输入数据
x = torch.randn(10)
y_true = torch.randn(1)
print(f'y_true: {y_true}')

# 前向传播
output = model_no_resnet(x)

# 计算损失
loss = criterion(output, y_true)

# 反向传播
optimizer.zero_grad()
loss.backward()

# 查看每一层的梯度
for name, param in model_no_resnet.named_parameters():
    print(f"{name} gradient: {param.grad.norm()}")


y_true: tensor([-0.7289])
layer1.weight gradient: 1.1623672246932983
layer1.bias gradient: 0.29009246826171875
layer2.weight gradient: 2.317469835281372
layer2.bias gradient: 0.63028883934021
layer3.weight gradient: 2.4428350925445557
layer3.bias gradient: 1.3776038885116577


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# 一个包含残差连接的神经网络
class ResNetNN(nn.Module):
    def __init__(self):
        super(ResNetNN, self).__init__()
        self.layer1 = nn.Linear(10, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, 1)

    def forward(self, x):
        residual = x  # 保存输入x，用于残差连接
        
        # 第一层
        x = torch.relu(self.layer1(x))
        
        # 检查输入和输出形状是否一致，不一致时使用线性层调整形状
        if residual.size(0) != x.size(0):  # 如果不匹配
            residual = self.layer1(residual)  # 使用layer1调整形状
        
        x = x + residual  # 残差连接：输入和输出相加
        residual = x  # 更新残差
        
        # 第二层
        x = torch.relu(self.layer2(x))
        
        # 再次检查形状并调整
        if residual.size(0) != x.size(0):
            residual = self.layer2(residual)
        
        x = x + residual  # 再次加入残差连接
        residual = x  # 更新残差
        
        # 第三层
        x = self.layer3(x)
        return x

# 初始化模型、损失函数和优化器
model_resnet = ResNetNN()
optimizer = optim.SGD(model_resnet.parameters(), lr=0.01)
criterion = nn.MSELoss()  # 选择均方误差损失函数

# 生成随机输入数据
x = torch.randn(10)  # 输入张量的大小是(10,)
y_true = torch.randn(1)  # 输出的大小是(1,)
print(f'y_true: {y_true}')

print(f'x: {x}')

# 前向传播
output = model_resnet(x)
print(f'output: {output}')

# 计算损失
loss = criterion(output, y_true)

# 反向传播
optimizer.zero_grad()
loss.backward()

# 查看每一层的梯度
for name, param in model_resnet.named_parameters():
    print(f"{name} gradient: {param.grad.norm()}")


y_true: tensor([-1.0424])
x: tensor([-0.2447, -0.3105, -0.6412,  0.3816, -0.6679, -1.6251, -1.8581,  0.5695,
        -1.6169, -1.2088])
output: tensor([0.6046], grad_fn=<ViewBackward0>)
layer1.weight gradient: 10.841142654418945
layer1.bias gradient: 3.1758835315704346
layer2.weight gradient: 10.468117713928223
layer2.bias gradient: 1.5394114255905151
layer3.weight gradient: 27.253820419311523
layer3.bias gradient: 3.293879508972168
